In [43]:
import soundcloud
import pandas as pd
import time
from ast import literal_eval
from collections import MutableMapping

from bs4 import BeautifulSoup
import requests
import urllib
import csv
import numpy as np
import psycopg2

In [2]:
c_id = '887b335a80f3e625454ebca548c53d96'
client = soundcloud.Client(client_id=c_id)

In [3]:
df = pd.read_csv('../data/sc_aliases.csv')

In [4]:
df.head()

,id,artist_name,sc_alias
0,1,009 Sound System,No Soundcloud Found
1,2,2 Unlimited,x
2,3,49ers,joe-castellano
3,4,808 State,No Soundcloud Found
4,5,A-Trak,a-trak


In [6]:
df = df[(df['sc_alias']!= 'x') & (df['sc_alias'] != 'No Soundcloud Found')]

In [57]:
artists = df['artist_name'].values

In [54]:
aliases = df['sc_alias'].values

In [58]:
artist_alias = zip(artists, aliases)

In [8]:
alias = 'a-trak'

In [9]:
url = 'https://soundcloud.com/' + alias + '/followers'

In [59]:
url = 'https://soundcloud.com/' + alias

## Connect to db

In [44]:
conn = psycopg2.connect(dbname='soundcloud', user='postgres')

In [45]:
c = conn.cursor()

## Create a table

In [51]:
query = """
CREATE TABLE sc_artists (
    id SERIAL PRIMARY KEY
    ,artist_name VARCHAR
    ,sc_alias VARCHAR
    ,city VARCHAR
    ,country VARCHAR
    ,followers_count VARCHAR
    ,followings_count VARCHAR
    ,last_modified VARCHAR
    ,playlist_count VARCHAR
    ,plan VARCHAR
    ,public_favorites_count VARCHAR
    ,track_count VARCHAR
);
"""

In [52]:
c.execute(query)

In [53]:
conn.commit()

In [56]:
query = """
INSERT INTO
    sc_artists
    (artist_name, sc_alias, city, country, followers_count, followings_count, last_modified, playlist_count, plan, public_favorites_count, track_count)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [71]:
def get_artist_info(artist_alias):
    total = str(len(artist_alias))
    for index, artist_alias in enumerate(artist_alias):
        try:
            artist = artist_alias[0]
            alias = artist_alias[1]
            url = 'https://soundcloud.com/' + alias
            artist_info = client.get('/resolve', url=url, linked_partitioning=1, limit=200).fields()
            c.execute(query, (str(artist), str(alias), artist_info['city'], artist_info['country'], str(artist_info['followers_count']), str(artist_info['followings_count']), str(artist_info['last_modified']), str(artist_info['playlist_count']), artist_info['plan'], str(artist_info['public_favorites_count']), str(artist_info['track_count']))) 
            conn.commit()
            if index % 25 == 0:
                print 'Artist {} of {} complete ({})'.format(str(index), total, artist)
        except Exception as e:
            conn.commit()
            print 'Artist {} of {} failed ({}) due to exception: {}'.format(str(index), total, artist, str(e))

In [73]:
conn.commit()

In [ ]:
get_artist_info(artist_alias)

Artist 0 of 7222 complete (49ers)
Artist 25 of 7222 complete (The Aston Shuffle)


In [ ]:
"""
for each follower,
grab the following fields:
city, country, followers_count, followings_count, last_modified, playlist_count, plan, public_favorites_count, track_count, write to CSV
"""

In [40]:
url

'https://soundcloud.com/a-trak'

In [36]:
page_size=200

In [60]:
followers = client.get('/resolve', url=url, linked_partitioning=1, limit=page_size)

In [63]:
followers.fields()

5910775

In [39]:
followers.obj['followers_count']

5910768

In [26]:
followers.keys()

[u'next_href', u'collection']

In [31]:
followers.obj.

[u'next_href', u'collection']

In [34]:
len(followers.obj['collection'])

50

In [ ]:
for alias in aliases[0:5]:
    url = 'https://soundcloud.com/' + alias + '/followers'
    try:
        followers = client.get('/resolve', url=url, linked_partitioning=1, limit=page_size)
        fols = [fol['username'] for fol in followers.obj['collection']]
        url = followers.obj.get('next_href')
        while url is not None:
            followers = client.get(url.split('api.soundcloud.com')[1], linked_partitioning=1, limit=page_size)
            fols.extend([fol['username'] for fol in followers.obj['collection']])
            url = followers.obj.get('next_href')
        all_followers.extend(zip(([alias for fol in xrange(len(fols))], fols)))
    except:
        continue

In [ ]:
for alias in aliases[0:5]:
    url = 'https://soundcloud.com/' + alias + '/followers'
    try:
        followers = client.get('/resolve', url=url, linked_partitioning=1, limit=page_size)
        for fol in followers.obj['collection']:
            folls = folls.append({'alias': alias, 'follower': fol['username']}, ignore_index=True)
        url = followers.obj.get('next_href')
        while url is not None:
            followers = client.get(url.split('api.soundcloud.com')[1], linked_partitioning=1, limit=page_size)
            for fol in followers.obj['collection']:
                folls = folls.append({'alias': alias, 'follower': fol}, ignore_index=True)
                url = followers.obj.get('next_href')
        print alias + ' worked'
    except Exception, e:
        print alias + ' did not work', str(e)
        continue